# Data Loading + Cleaning + Merging

In [2]:
# import packages
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch, Circle
import matplotlib as mpl

import warnings
warnings.filterwarnings('ignore')

In [3]:
# load dataset
df = pd.read_csv("beam_dashboard.csv")

In [4]:
# preview dataset
df.head()

,Year,Month,State,Source,Pathogen,Serotype/Species,Number of isolates,Outbreak associated isolates,New multistate outbreaks,New multistate outbreaks - US,% Isolates with clinically important antimicrobial resistance,Number of sequenced isolates analyzed by NARMS
0,2024,1,AK,Stool,Campylobacter,jejuni,1,NaN,NaN,NaN,NaN,NaN
1,2024,1,AL,Stool,Campylobacter,coli,1,NaN,NaN,NaN,NaN,NaN
2,2024,1,AL,Stool,Campylobacter,jejuni,2,NaN,NaN,NaN,NaN,NaN
3,2024,1,AR,Stool,Campylobacter,jejuni,1,NaN,NaN,NaN,NaN,NaN
4,2024,1,CA,Stool,Campylobacter,jejuni,1,NaN,NaN,NaN,NaN,NaN


In [5]:
# drop columns to the right of 'Number of isolates' due to tons of missing / N/A data
df = df.drop(['Outbreak associated isolates', 
              'New multistate outbreaks', 
              'New multistate outbreaks - US', 
              '% Isolates with clinically important antimicrobial resistance',
              'Number of sequenced isolates analyzed by NARMS'], axis = 1)
df.head()

,Year,Month,State,Source,Pathogen,Serotype/Species,Number of isolates
0,2024,1,AK,Stool,Campylobacter,jejuni,1
1,2024,1,AL,Stool,Campylobacter,coli,1
2,2024,1,AL,Stool,Campylobacter,jejuni,2
3,2024,1,AR,Stool,Campylobacter,jejuni,1
4,2024,1,CA,Stool,Campylobacter,jejuni,1


In [6]:
# rename columns
df.columns = ['year', 'month', 'state', 'source', 'pathogen', 'serotype_species', 'num_isolates']
df.head()

,year,month,state,source,pathogen,serotype_species,num_isolates
0,2024,1,AK,Stool,Campylobacter,jejuni,1
1,2024,1,AL,Stool,Campylobacter,coli,1
2,2024,1,AL,Stool,Campylobacter,jejuni,2
3,2024,1,AR,Stool,Campylobacter,jejuni,1
4,2024,1,CA,Stool,Campylobacter,jejuni,1


In [7]:
# check for missing values
df.isna().sum()

year                0
month               0
state               5
source              0
pathogen            0
serotype_species    0
num_isolates        0
dtype: int64

In [8]:
# drop rows where state information is not available, messes up geospatial data later on
df = df.dropna()
# verify that there is no more missing data
df.isna().sum()

year                0
month               0
state               0
source              0
pathogen            0
serotype_species    0
num_isolates        0
dtype: int64

In [9]:
df['source'].unique()

array(['Stool', 'Blood', 'Other', 'Urine'], dtype=object)

In [10]:
shapefile_path = ("tl_2024_us_state/tl_2024_us_state.shp")
gdf = gpd.read_file(shapefile_path)


In [11]:
# tutorial for mapping from Oscar Leo in DEV

# define features
edge_color = "#30011E"

sns.set_style({
    "font.family": "DejaVu Sans"
})

In [12]:
# remove “unincorporated territories” such as Guam, Puerto Rico, and American Samoa
states = gdf[~gdf.STATEFP.isin(["72", "69", "60", "66", "78"])]

# adjust the state mapping projection
states = states.to_crs("ESRI:102003")

In [13]:
# translating data to draw Alaska and Hawaii under mainland USA (just for aesthetic purposes!)
def translate_geometries(df, x, y, scale, rotate):
    df.loc[:, "geometry"] = df.geometry.translate(yoff = y, xoff = x)
    center = df.dissolve().centroid.iloc[0]
    df.loc[:, "geometry"] = df.geometry.scale(xfact = scale, yfact = scale, origin = center)
    df.loc[:, "geometry"] = df.geometry.rotate(rotate, origin = center)
    return df

In [14]:
# serparates Hawaii and Alaska to adjust their geometries
def adjust_maps(df):
    df_main_land = df[~df.STATEFP.isin(["02", "15"])]
    df_alaska = df[df.STATEFP == "02"]
    df_hawaii = df[df.STATEFP == "15"]
    
    # putting them back into new dataframes
    df_alaska = translate_geometries(df_alaska, 1300000, -4900000, 0.5, 32)
    df_hawaii = translate_geometries(df_hawaii, 5400000, -1500000, 1, 24)

    return pd.concat([df_main_land, df_alaska, df_hawaii])

In [15]:
# adding adjust_maps to the code
states = adjust_maps(states)

In [16]:
# merging shapefiles to the data
merged_df = df.merge(states, left_on='state', right_on='STUSPS')

In [17]:
merged_df.head()

,year,month,state,source,pathogen,serotype_species,num_isolates,REGION,DIVISION,STATEFP,...,STUSPS,NAME,LSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,2024,1,AK,Stool,Campylobacter,jejuni,1,4,9,02,...,AK,Alaska,00,G4000,A,1479508971743,244710526650,+63.3473560,-152.8397334,"MULTIPOLYGON (((-2778499.467 -1620603.792, -27..."
1,2024,1,AL,Stool,Campylobacter,coli,1,3,6,01,...,AL,Alabama,00,G4000,A,131185561946,4581813708,+32.7395785,-086.8434469,"POLYGON ((1023282.917 -582853.454, 1023269.168..."
2,2024,1,AL,Stool,Campylobacter,jejuni,2,3,6,01,...,AL,Alabama,00,G4000,A,131185561946,4581813708,+32.7395785,-086.8434469,"POLYGON ((1023282.917 -582853.454, 1023269.168..."
3,2024,1,AR,Stool,Campylobacter,jejuni,1,3,7,05,...,AR,Arkansas,00,G4000,A,134658517854,3122715710,+34.8955256,-092.4446262,"POLYGON ((461603.731 -366288.124, 461727.959 -..."
4,2024,1,CA,Stool,Campylobacter,jejuni,1,4,9,06,...,CA,California,00,G4000,A,403673433805,20291632828,+37.1551773,-119.5434183,"MULTIPOLYGON (((-1976277.366 663684.581, -1976..."


In [18]:
# keeping certain columns for EDA
merged_df = merged_df[['year', 'month', 'state', 'source', 'pathogen', 'serotype_species', 'num_isolates', 'geometry']]
merged_df.head()

,year,month,state,source,pathogen,serotype_species,num_isolates,geometry
0,2024,1,AK,Stool,Campylobacter,jejuni,1,"MULTIPOLYGON (((-2778499.467 -1620603.792, -27..."
1,2024,1,AL,Stool,Campylobacter,coli,1,"POLYGON ((1023282.917 -582853.454, 1023269.168..."
2,2024,1,AL,Stool,Campylobacter,jejuni,2,"POLYGON ((1023282.917 -582853.454, 1023269.168..."
3,2024,1,AR,Stool,Campylobacter,jejuni,1,"POLYGON ((461603.731 -366288.124, 461727.959 -..."
4,2024,1,CA,Stool,Campylobacter,jejuni,1,"MULTIPOLYGON (((-1976277.366 663684.581, -1976..."


In [19]:
merged_df.head()

,year,month,state,source,pathogen,serotype_species,num_isolates,geometry
0,2024,1,AK,Stool,Campylobacter,jejuni,1,"MULTIPOLYGON (((-2778499.467 -1620603.792, -27..."
1,2024,1,AL,Stool,Campylobacter,coli,1,"POLYGON ((1023282.917 -582853.454, 1023269.168..."
2,2024,1,AL,Stool,Campylobacter,jejuni,2,"POLYGON ((1023282.917 -582853.454, 1023269.168..."
3,2024,1,AR,Stool,Campylobacter,jejuni,1,"POLYGON ((461603.731 -366288.124, 461727.959 -..."
4,2024,1,CA,Stool,Campylobacter,jejuni,1,"MULTIPOLYGON (((-1976277.366 663684.581, -1976..."


# EDA

In [35]:
counts_per_year = merged_df.groupby(['year', 'pathogen']).size().unstack(fill_value=0)
